In [1]:
!python --version

Python 3.6.13 :: Anaconda, Inc.


In [ ]:
#installation
# %pip install joblib
# %pip install tqdm
# %pip install scikit-learn
# %pip install nltk

In [2]:
# imports
import joblib
import sklearn
import random
import pandas as pd
import numpy as np
from tqdm import tqdm  
from bert_embedding import BertEmbedding
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity
# import matplotlib.pyplot as plt
# import seaborn as sn

In [3]:
# NLTK
import nltk 
import re
import string 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
stop_w = stopwords.words('english')
bert_embedding = BertEmbedding(max_seq_length=50)

In [6]:
# loading dataset
df = pd.read_csv("C:\\Users\\harsh\\OneDrive\\Desktop\\Mental Health Chatbot\\chatbot dataset.csv",encoding="utf-8")

In [ ]:
df.head

In [7]:
df2 = df #copy

In [8]:
# pre-processing
import string
stop_w = stopwords.words('english')
def clean(column,df,stopwords=False):
  #remove stop words
  df[column] = df[column].apply(str)
  df[column] = df[column].str.lower().str.split()
  if stopwords:
        df[column]=df[column].apply(lambda x: [item for item in x if item not in stop_w])
  #remove punctuation
  df[column]=df[column].apply(lambda x: [item for item in x if item not in string.punctuation])
  df[column]=df[column].apply(lambda x: " ".join(x))
  
  #lowercase text - already done in models

In [9]:
clean('questionText',df2) #kept stopwords

In [13]:
# extracting questions_embeddings
questions = df2["questionText"].to_list()
questions_embeddings = bert_embedding(questions)

In [14]:
#storing the questions_embeddings in a list
sent_bertphrase_embeddings = []
for emb in questions_embeddings:
  sent_bertphrase_embeddings.append(emb[1])

In [10]:
clean('answerText',df2,stopwords=True)   #kept stopwords in ans column

In [16]:
# extracting answers_embeddings
answers = df2["answerText"].to_list()
answers_embeddings = bert_embedding(answers)

In [17]:
np.array(answers_embeddings).shape

(2129, 2)

In [18]:
# storing the answers_embeddings in a list
sent_bertphrase_ans_embeddings = []
for emb in answers_embeddings:
  sent_bertphrase_ans_embeddings.append(emb[1])

In [19]:
# converting to pickle file (for questions_embeddings)
joblib.dump(sent_bertphrase_embeddings,"questions.pkl")

['questions.pkl']

In [20]:
# converting to pickle file (for answers_embeddings)
joblib.dump(sent_bertphrase_ans_embeddings,"answers.pkl")

['answers.pkl']

In [ ]:
# using bert word embeddings to encode the questions
sent_bertphrase_embeddings=[]
for index,row in tqdm(df2.iterrows()):
  sent = row['questionText']
  #sent_bertphrase_embeddings.append(bc.encode([sent]));
  result = bert_embedding(sent)
  sent_bertphrase_embeddings.append(result)
#   aborted

In [23]:
def get_cleaned_sentences(df):    
    cleaned_sentences=[]
    for index,row in df.iterrows():
        cleaned = row['questionText']
        cleaned_sentences.append(cleaned);
    return cleaned_sentences

In [24]:
cleaned_sentences = get_cleaned_sentences(df2)

In [23]:
df2.iloc[1][8]

'hello, thank question seeking advice this. feelings worthlessness unfortunately common. fact, people, all, felt degree point life. alone. changing feelings like changing thoughts hard do. minds amazing minute change thought another one right take place. without permission, another thought pop there. new thought may feel worse last one! guess tried several things improve even reaching here. people often try thinking positive thoughts, debating thoughts, simply telling need "snap it" also thought carries self-criticism. people try different approach, counselors help this. idea instead trying change thoughts, change respond them. learn skills allow manage difficult thoughts feelings differently impact right now. people, actually begin experience less hurtful thoughts learn manage ones differently. acceptance commitment therapy may good choice you. information online even self-help books use teach skills mentioned. skills, require practice, many people found great relief enriched life lea

In [25]:
# loading questions pickle file
sent_bertphrase_embeddings = joblib.load("C:\\Users\\harsh\\OneDrive\\Desktop\\Mental Health Chatbot\\questions.pkl")

In [26]:
# loading answers pickle file
sent_bertphrase_ans_embeddings = joblib.load("C:\\Users\\harsh\\OneDrive\\Desktop\\Mental Health Chatbot\\answers.pkl")

In [26]:
import json
import requests

In [71]:
# getting hold of embeddings value from user input text
def get_embeddings(texts):
    url = 'https://4900-203-194-107-244.ngrok.io'
    headers = {
        'content-type':'application/json'
    }
    data = {
        "id":123,
        "texts":texts,
        "is_tokenized": False
    }
    data = json.dumps(data)
    # print(data)
    r = requests.post(url, data=data, headers=headers).json()
    return (r['result'])

In [74]:
get_embeddings("Hello there")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [28]:
# def retrieve(sent_bertphrase_embeddings,example_query):
#     max_=-1
#     max_i = -1
#     for index,emb in enumerate(sent_bertphrase_embeddings):
#         sim_score = cosine_similarity(emb,example_query)[0][0]
#         if sim_score>max_:
#             max_=sim_score
#             max_i=index
#     print("\n");
#     print("Retrieved: "+str(max_)+" ",df.iloc[max_i,3])  # 3 is index for q text
#     print(df.iloc[max_i,8])    # 8 is the index for the answer text  
#     return str(df.iloc[max_i,8])

In [14]:
# testing sample
data = ["I have been feeling really down. I do not feel like eating and I feel miserable.",
         "I've been really stressed out. I just broke up with my boyfriend. Our relationship had problems",
        "My boss has been troubling me. I am sick of it.",
       "He keeps hitting me at home. He is abusive and throws things and yells at me. Husband is drunk all the time.",
        "Im bored and I like to play with toys and cars",
        "I just lost my grandparents. They both passed away this year and I am unable to cope and function with this loss.",
        "Hey! how are you? yo whatsup",
        "I was in the army and I just got out of the war. I think I'm having severe PTSD",
        "How can I deal with OCD?",
        "My partner snores a lot and I can't sleep"]
x_try = pd.DataFrame(data,columns=['text'])

In [15]:
x_try

,text
0,I have been feeling really down. I do not feel...
1,I've been really stressed out. I just broke up...
2,My boss has been troubling me. I am sick of it.
3,He keeps hitting me at home. He is abusive and...
4,Im bored and I like to play with toys and cars
5,I just lost my grandparents. They both passed ...
6,Hey! how are you? yo whatsup
7,I was in the army and I just got out of the wa...
8,How can I deal with OCD?
9,My partner snores a lot and I can't sleep


In [31]:
# getting hold of a certain question
question = x_try.iloc[7][0]
question

"I was in the army and I just got out of the war. I think I'm having severe PTSD"

In [16]:
# extracting embedding values
x_try_list = x_try['text'].to_list()
x_try_embeddings = bert_embedding(x_try_list)

In [17]:
# storing embeddings in a list (here as taking a eg, consider alternative of dumping step into pickle file)
user_queries = []
for x in x_try_embeddings:
  user_queries.append(x[1])

example_query = user_queries[0]
example_question = sent_bertphrase_embeddings[0]

In [19]:
def retrieve(sent_bertphrase_embeddings,example_query):
  max_=-1
  max_i = -1
  for index,emb in enumerate(sent_bertphrase_embeddings):
    sim_score = cosine_similarity(emb,example_query)[0][0]
    if sim_score>max_:
      max_=sim_score
      max_i=index
  print("\n");
  print("Retrieved: "+str(max_)+" ",df.iloc[max_i,3])  # 3 is index for q text
  print(df.iloc[max_i,8])    # 8 is the index for the answer text  
  return str(df.iloc[max_i,8])
#sim_score = cosine_similarity(example_question, example_query)[0][0]

In [37]:
# retrieving similar queries with that to example query
for queries in range(len(user_queries)):
  print(x_try.iloc[queries][0])
  example_query = user_queries[queries]
  retrieve(sent_bertphrase_embeddings,example_query)

I have been feeling really down. I do not feel like eating and I feel miserable.


Retrieved: 0.9189749  i have been feeling more and more down for over a month. i have started having trouble sleeping due to panic attacks, but they are almost never triggered by something that i know of.
answers inner lives successfully reached sense feeling grounded oneself.first step accept nervousness restless sleep. often possible, sleep daytimes order body catch need rest.accept feeling down. normal feel while. place self-acceptance, trust answers come mind. often answers complicated topics come small pieces, whole unit.also, description panic attacks also completely normal. often arise unrelated particular conditions given moment. healthy symptom body trying expel bad feelings anxiety erupt times.so, self-acceptance, tolerance process clearing worn emotional clutter, sleep odd times possible, ways stabilize yourself, also feel calm good!
I've been really stressed out. I just broke up with my boyfr

In [29]:
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf): #USE BOTH QUESTION AND ANSWER EMBEDDINGS FOR CS
    max_sim=-1
    index_sim=-1
    valid_ans = []
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0]
        #print(index, sim, sentences[index])
        if sim>=max_sim:
            max_sim=sim
            index_sim=index
            valid_ans.append(index_sim) #get all possible valid answers with same confidence

    #Calculate answer with highest cosine similarity 
    max_a_sim=-1
    answer=""
    for ans in valid_ans:
      answer_text = FAQdf.iloc[ans,8] #answer 
      answer_em = sent_bertphrase_ans_embeddings[ans] #get embedding from index
      similarity = cosine_similarity(answer_em,question_embedding)[0][0]
      if similarity>max_a_sim:
        max_a_sim = similarity
        answer = answer_text
    #print("\n")
    #print("Question: ",question)
    #print("\n");
    #print("Retrieved: "+str(max_sim)+" ",FAQdf.iloc[index_sim,3])  # 3 is index for q text
    #print(FAQdf.iloc[index_sim,8])    # 8 is the index for the answer text 
    #check confidence level
    if max_a_sim<0.70: 
        return "Could you please elaborate your situation more? I don't really understand." 
    return answer  
    #print(answer)
     

In [ ]:
for index,row in x_try.iterrows():
  question = row['text']
  print(question)
  question_embedding = list(bert_embedding(question))
  # question_embedding = get_embeddings([question])
  retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df)

In [ ]:
for index,row in x_try.iterrows():
  example_query = row['text']
  print(example_query)
  example_query_emb = get_embeddings([example_query])
  ans = retrieve(sent_bertphrase_embeddings,example_query_emb)

In [42]:
%pip install textblob
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [43]:
from textblob import TextBlob
from textblob.sentiments import *
blob = TextBlob("hmm fine ig", analyzer=PatternAnalyzer())
blob.sentiment.polarity

0.4166666666666667